In [1]:
import pandas as pd
import numpy as np
import string
import pickle
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
data_path = './data/twitter_MBTI.csv'


data = pd.read_csv(data_path,sep=",")

In [3]:
import pandas as pd
import numpy as np
import string
import pickle
import nltk
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords.remove('no')
nltk_stopwords.remove('not')

[nltk_data] Downloading package punkt to /storage/hhbao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /storage/hhbao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /storage/hhbao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /storage/hhbao/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /storage/hhbao/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
punctuation_translator = str.maketrans('', '', string.punctuation)

In [6]:
def preprocess_text(s, lowercase=True, remove_stopwords=True, remove_punctuation=True, stemmer=None, lemmatizer=None):
    tokens = word_tokenize(s)

    if lemmatizer is not None:
        tokens = lemmatize_tokens(lemmatizer, tokens)
    elif stemmer is not None:
        tokens = stem_tokens(stemmer, tokens)

    if lowercase:
        tokens = [token.lower() for token in tokens]
    
    if remove_stopwords:
        tokens = [token for token in tokens if not token in nltk_stopwords]
    
    # Remove all punctuation marks if needed (note: also converts, e.g, "Mr." to "Mr")
    if remove_punctuation:
        tokens = [ ''.join(c for c in s if c not in string.punctuation) for s in tokens ]
        tokens = [ token for token in tokens if len(token) > 0 ] # Remove "empty" tokens

    # if (len(tokens) == 0):
    #     print('len = 0: '+ s)

    return ' '.join(tokens)

def remove_punctuation(s):
    return s.translate(punctuation_translator)

def lemmatize_tokens(lemmatizer, tokens):
    pos_tag_list = pos_tag(tokens)
    for idx, (token, tag) in enumerate(pos_tag_list):
        tag_simple = tag[0].lower() # Converts, e.g., "VBD" to "c"
        if tag_simple in ['n', 'v', 'j']:
            word_type = tag_simple.replace('j', 'a') 
        else:
            word_type = 'n'
        lemmatized_token = lemmatizer.lemmatize(token, pos=word_type)
        tokens[idx] = lemmatized_token
    return tokens

def stem_tokens(stemmer, tokens):
    for idx, token in enumerate(tokens):
        tokens[idx] = stemmer.stem(token)
    return tokens

In [7]:
data['ie'] = data['label'].apply(lambda x: x[0])
data['ns'] = data['label'].apply(lambda x: x[1])
data['ft'] = data['label'].apply(lambda x: x[2])
data['pj'] = data['label'].apply(lambda x: x[3])


train_data, test_and_valid_data = train_test_split(data, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(test_and_valid_data, test_size=0.5, random_state=42)
vectorizer = TfidfVectorizer(lowercase=False)

content = train_data.iloc[:,1]
content = [ preprocess_text(x, remove_stopwords=True, remove_punctuation=True, lemmatizer=wordnet_lemmatizer) for x in content ]

X = vectorizer.fit_transform(content)
Y_i = train_data.iloc[:,3]
Y_n = train_data.iloc[:,4]
Y_f = train_data.iloc[:,5]
Y_p = train_data.iloc[:,6]


Y_test_i = test_data.iloc[:,3]
Y_test_n = test_data.iloc[:,4]
Y_test_f = test_data.iloc[:,5]
Y_test_p = test_data.iloc[:,6]
X_test_content = test_data.iloc[:,1]
X_test_content = [ preprocess_text(x, remove_stopwords=True, remove_punctuation=True, lemmatizer=wordnet_lemmatizer) for x in X_test_content ]

X_test = vectorizer.transform(X_test_content)

## LR

In [8]:
lr_classifier = LogisticRegression(multi_class = 'multinomial',solver='newton-cg',max_iter = 1000).fit(X, Y_i)
Y_lr_predict = lr_classifier.predict(X_test)
print(classification_report(Y_test_i, Y_lr_predict, digits=4))

              precision    recall  f1-score   support

           e     0.6071    0.1283    0.2118       265
           i     0.6818    0.9574    0.7965       517

    accuracy                         0.6765       782
   macro avg     0.6445    0.5429    0.5041       782
weighted avg     0.6565    0.6765    0.5983       782



In [9]:
lr_classifier = LogisticRegression(multi_class = 'multinomial',solver='newton-cg',max_iter = 1000).fit(X, Y_n)
Y_lr_predict = lr_classifier.predict(X_test)
print(classification_report(Y_test_n, Y_lr_predict, digits=4))

              precision    recall  f1-score   support

           n     0.7913    0.9984    0.8829       619
           s     0.0000    0.0000    0.0000       163

    accuracy                         0.7903       782
   macro avg     0.3956    0.4992    0.4414       782
weighted avg     0.6264    0.7903    0.6988       782



In [10]:
lr_classifier = LogisticRegression(multi_class = 'multinomial',solver='newton-cg',max_iter = 1000).fit(X, Y_f)
Y_lr_predict = lr_classifier.predict(X_test)
print(classification_report(Y_test_f, Y_lr_predict, digits=4))

              precision    recall  f1-score   support

           f     0.6656    0.8681    0.7535       470
           t     0.6331    0.3429    0.4449       312

    accuracy                         0.6586       782
   macro avg     0.6494    0.6055    0.5992       782
weighted avg     0.6526    0.6586    0.6304       782



In [11]:
lr_classifier = LogisticRegression(multi_class = 'multinomial',solver='newton-cg',max_iter = 1000).fit(X, Y_p)
Y_lr_predict = lr_classifier.predict(X_test)
print(classification_report(Y_test_p, Y_lr_predict, digits=4))

              precision    recall  f1-score   support

           j     0.6915    0.5718    0.6260       341
           p     0.7080    0.8027    0.7524       441

    accuracy                         0.7020       782
   macro avg     0.6997    0.6873    0.6892       782
weighted avg     0.7008    0.7020    0.6973       782



## NB

In [12]:
# Multinomial Naive Bayes 
mnb_classifier = MultinomialNB().fit(X, Y_i)
Y_predict = mnb_classifier.predict(X_test)
print(classification_report(Y_test_i, Y_predict, digits=4))

              precision    recall  f1-score   support

           e     0.0000    0.0000    0.0000       265
           i     0.6611    1.0000    0.7960       517

    accuracy                         0.6611       782
   macro avg     0.3306    0.5000    0.3980       782
weighted avg     0.4371    0.6611    0.5263       782



/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [13]:
# Multinomial Naive Bayes 
mnb_classifier = MultinomialNB().fit(X, Y_n)
Y_predict = mnb_classifier.predict(X_test)
print(classification_report(Y_test_n, Y_predict, digits=4))

              precision    recall  f1-score   support

           n     0.7916    1.0000    0.8837       619
           s     0.0000    0.0000    0.0000       163

    accuracy                         0.7916       782
   macro avg     0.3958    0.5000    0.4418       782
weighted avg     0.6266    0.7916    0.6995       782



/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [14]:
# Multinomial Naive Bayes 
mnb_classifier = MultinomialNB().fit(X, Y_f)
Y_predict = mnb_classifier.predict(X_test)
print(classification_report(Y_test_f, Y_predict, digits=4))

              precision    recall  f1-score   support

           f     0.6021    0.9979    0.7510       470
           t     0.6667    0.0064    0.0127       312

    accuracy                         0.6023       782
   macro avg     0.6344    0.5021    0.3818       782
weighted avg     0.6278    0.6023    0.4564       782



In [15]:
# Multinomial Naive Bayes 
mnb_classifier = MultinomialNB().fit(X, Y_p)
Y_predict = mnb_classifier.predict(X_test)
print(classification_report(Y_test_p, Y_predict, digits=4))

              precision    recall  f1-score   support

           j     0.8235    0.0411    0.0782       341
           p     0.5725    0.9932    0.7264       441

    accuracy                         0.5780       782
   macro avg     0.6980    0.5171    0.4023       782
weighted avg     0.6820    0.5780    0.4437       782



## Decision Tree

In [16]:
# Decision Tree
tree_classifier = tree.DecisionTreeClassifier().fit(X, Y_i)
Y_tree_predict = tree_classifier.predict(X_test)
print(classification_report(Y_test_i, Y_tree_predict, digits=4))

              precision    recall  f1-score   support

           e     0.5830    0.5962    0.5896       265
           i     0.7906    0.7814    0.7860       517

    accuracy                         0.7187       782
   macro avg     0.6868    0.6888    0.6878       782
weighted avg     0.7203    0.7187    0.7194       782



In [17]:
# Decision Tree
tree_classifier = tree.DecisionTreeClassifier().fit(X, Y_n)
Y_tree_predict = tree_classifier.predict(X_test)
print(classification_report(Y_test_n, Y_tree_predict, digits=4))

              precision    recall  f1-score   support

           n     0.8633    0.8368    0.8499       619
           s     0.4451    0.4969    0.4696       163

    accuracy                         0.7660       782
   macro avg     0.6542    0.6669    0.6597       782
weighted avg     0.7761    0.7660    0.7706       782



In [18]:
# Decision Tree
tree_classifier = tree.DecisionTreeClassifier().fit(X, Y_f)
Y_tree_predict = tree_classifier.predict(X_test)
print(classification_report(Y_test_f, Y_tree_predict, digits=4))

              precision    recall  f1-score   support

           f     0.7336    0.7617    0.7474       470
           t     0.6190    0.5833    0.6007       312

    accuracy                         0.6905       782
   macro avg     0.6763    0.6725    0.6740       782
weighted avg     0.6879    0.6905    0.6888       782



In [19]:
# Decision Tree
tree_classifier = tree.DecisionTreeClassifier().fit(X, Y_p)
Y_tree_predict = tree_classifier.predict(X_test)
print(classification_report(Y_test_p, Y_tree_predict, digits=4))

              precision    recall  f1-score   support

           j     0.6295    0.6628    0.6457       341
           p     0.7281    0.6984    0.7130       441

    accuracy                         0.6829       782
   macro avg     0.6788    0.6806    0.6793       782
weighted avg     0.6851    0.6829    0.6836       782



# BERT

In [20]:
# Bert
import pandas as pd
import torch
import numpy as np
import transformers
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {
        'i': 0,
        'e': 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['ie']]
        # self.labels = [label for label in df['Label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]
    # Get the labels
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    # Get the batch's labels
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

/storage_fast/hhbao/anaconda/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [22]:
def train(model, train_data, val_data, learning_rate, epochs):

    # Data Preparation
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    # Cuda
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # Loss & Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [23]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [24]:
EPOCHS = 10 
np.random.seed(42)
model = BertClassifier()
LR = 1e-6
              
train(model, train_data, valid_data, LR, EPOCHS)

100%|██████████| 196/196 [02:51<00:00,  1.14it/s]


Epochs: 1 | Train Loss:  0.021 | Train Accuracy:  0.662 | Val Loss:  0.020 | Val Accuracy:  0.670


100%|██████████| 196/196 [02:53<00:00,  1.13it/s]


Epochs: 2 | Train Loss:  0.020 | Train Accuracy:  0.670 | Val Loss:  0.021 | Val Accuracy:  0.673


100%|██████████| 196/196 [02:53<00:00,  1.13it/s]


...

Epochs: 8 | Train Loss:  0.019 | Train Accuracy:  0.673 | Val Loss:  0.021 | Val Accuracy:  0.670


100%|██████████| 196/196 [02:52<00:00,  1.13it/s]


Epochs: 9 | Train Loss:  0.019 | Train Accuracy:  0.676 | Val Loss:  0.021 | Val Accuracy:  0.659


100%|██████████| 196/196 [02:52<00:00,  1.13it/s]


Epochs: 10 | Train Loss:  0.019 | Train Accuracy:  0.675 | Val Loss:  0.022 | Val Accuracy:  0.671


In [25]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [26]:
evaluate(model, test_data)


Test Accuracy:  0.660


In [27]:
EPOCHS = 2
np.random.seed(42)
model = BertClassifier()
LR = 1e-6
              
train(model, train_data, valid_data, LR, EPOCHS)
evaluate(model, test_data)


100%|██████████| 196/196 [02:49<00:00,  1.16it/s]


Epochs: 1 | Train Loss:  0.021 | Train Accuracy:  0.652 | Val Loss:  0.021 | Val Accuracy:  0.672


100%|██████████| 196/196 [02:53<00:00,  1.13it/s]


Epochs: 2 | Train Loss:  0.020 | Train Accuracy:  0.673 | Val Loss:  0.021 | Val Accuracy:  0.671
Test Accuracy:  0.661
